In [ ]:
!pip install pyspark

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark.sql.functions as func
import sys
import os
from datetime import  datetime, timedelta
from pyspark.sql.types import ArrayType,DateType,StructType
from pyspark.sql.functions import when,col,lit
from pyspark.sql.window import Window
from pyspark.sql import Window
from pyspark.sql import functions as F

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Rossmann').getOrCreate()

In [ ]:
path      = "../input"
file_name = path + '/train.csv' 

In [ ]:
from pyspark.sql.types import (StructField, StringType, StructType, IntegerType,
                               FloatType, DateType) 

data_schema = [StructField('Store', StringType(), True),
               StructField('DayOfWeek', StringType(), True),
               StructField('Date', DateType(), True),
               StructField('Sales',FloatType(), True),
               StructField('Customers',StringType(), True),
               StructField('Open',StringType(), True),
               StructField('Promo',StringType(), True),
               StructField('StateHoliday',StringType(), True)]

In [ ]:
final_struc = StructType(fields = data_schema)
df = spark.read.csv(file_name,schema=final_struc)

In [ ]:
#Consider only active stores
df = df.where(df['Open'] == 1)
df.count()

In [ ]:
df.printSchema()

In [ ]:
# function for getting date range
def generate_date_series(start, stop):
    date_range = []
    no_of_days = (stop-start).days + 1
    for x in range(0, no_of_days):
        date_list = [start + timedelta(days=x)]
        k = date_list[0]
        date_range.append([k])
    return date_range


In [ ]:
from pyspark.sql.functions import count, isnan


**Choose Store 1**

In [ ]:
df = df.where(df['Store'] == 1)

In [ ]:
df.createOrReplaceTempView('storedata')

**Impute values for the slected store 1**

In [ ]:
result = spark.sql("SELECT MIN(Date) as mindate,MAX(Date) as maxdate from storedata ")
minim = result.head(5)[0][0] 
maxim = result.head(5)[0][1]

In [ ]:
minim = minim - timedelta(days=1)
date_list = generate_date_series(minim,maxim)


In [ ]:

df_datelist = spark.createDataFrame(date_list,['Date'])
df_datelist.show(5)

In [ ]:
df = df_datelist.join(df,["Date"],"leftouter").orderBy("Date")
df.head(5)

In [ ]:

pd_df = df.toPandas()


**Impute with forward and backward fill**

In [ ]:
pd_df = pd_df.fillna(method = 'bfill').fillna(method = 'ffill')

In [ ]:
#placing back to spark dataframe
df = spark.createDataFrame(pd_df)

In [ ]:
df.head(5)

In [ ]:
df.filter((df["Customers"] == "") | df["Customers"].isNull() | isnan(df["Customers"])).count()



In [ ]:
df.filter((df["Store"] == "") | df["Store"].isNull() | isnan(df["Store"])).count()

**Add week number coulmn to data frame**

In [ ]:
from pyspark.sql.functions import weekofyear,year
                                        


In [ ]:
df = df.withColumn("Week",weekofyear("Date"))

***Add the Year column***

In [ ]:
df= df.withColumn('Year', year('Date'))


In [ ]:
df.printSchema()

***Aggregating data to week level***

In [ ]:
#Change the Year data type to string
df = df.withColumn("Year", df["Year"].cast(StringType()))
df = df.withColumn("Store", df["Store"].cast(IntegerType()))


In [ ]:
df.printSchema()

df.createOrReplaceTempView('storedata')

In [ ]:
result = spark.sql("select  Year, Week, sum(Sales) as Tot_Sales from storedata \
                   group by Year, Week order by Year desc,Week desc ")

In [ ]:
result.head(5)

***Cumulative Sales for present, Week1, Week2, Week3***

In [ ]:


windowval = (Window.partitionBy('Year').orderBy(('Week'))
             .rangeBetween(Window.unboundedPreceding, 0))
df_w_cumsum = result.withColumn('cum_sum', F.sum('Tot_Sales').over(windowval))
df_w_cumsum.orderBy((df_w_cumsum['Year']).desc(),(df_w_cumsum['Week']).desc() ).show(4)
#df_w_cumsum.head(5)